# Demo 2: Bayesian Integration

In Demo 2, we will seek to better understand optimal Bayesian Integration.
The demo is split in two different experimental scenarios, which are explored in the second half of the lecture.
Each demo will be accompanied by **guiding questions**, which will help with the learning process as we go.
    
**Part A** illustrates a unisensory (proprioceptive) hand localization experiment with sensory input and a prior.

**Part B** illustrates a multisensory (proprioceptive & vision) hand localization experiment with sensory input and a prior.



## Demo 2A: Proprioceptive Hand Localization

In the current demo, we will explore Bayesian integration in the case of proprioceptive hand localization, 
where the perceived hand location (posterior), is derived from integrating proprioceptive input (likelihood)
and a prior distribution for hand location.

In the demo, the likelihood is illustrated with a red curve, the prior with a blue curve, and the posterior with a purple curve.

Two adjustable settings control the properties of the Likelihood distribution.

1. "P Stimulus" controls the location of the hand.
2. "P Noise σ" controls the proprioceptive noise.

Two adjustable settings control the properties of the Prior distribution.

1. "Prior" controls the mean of the prior distribution.
2. "Prior σ" controls the standard deviation of the prior distribution.

To make a measurement, click the box "Make Measurement". 
Doing so will randomly sample a sensory observation from the measurement distribution and illustrate the Likelihood distribution.

To derive a posterior, click the box "Show Posterior". 


### Guiding Questions
1. Let's first explore how the likelihood and the prior vary from measurement-to-measurement. Make several measurements using the "Make Measurement" box. 
How does taking a measurement affect the likelihood and the prior? What about if you change the mean locations and the noise/uncertainty levels?

2. Click to show the posterior distribution. What do you notice about it compared to the likelihood and the prior? 
If you change the noise and uncertainty levels, what does this do to the posterior? Explore several combinations.

3. How does changing the distance between the likelihood and prior means change the size of the bias?
                               

In [31]:
# ---------------------------------------------------------------------------
# Demo 2A

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Checkbox, VBox, HBox, Button, Output
from IPython.display import display

# Make static images sharp inside notebooks
%matplotlib inline
plt.rcParams["figure.dpi"] = 120

# Axis limits
P_RANGE = (-10, 10)
SIGMA_P_RANGE = (1, 5)
PR_RANGE = (-10, 10)
SIGMA_PR_RANGE = (1, 10)
XLIM = (-20, 20)
YLIM = (-0.002, 0.05)

# Output widget
output = Output()

# Controls
s_p_slider = FloatSlider(value=-7.0, min=P_RANGE[0], max=P_RANGE[1], step=0.1,
                         description="P Stimulus:", continuous_update=True)
sigma_p_slider = FloatSlider(value=3.0, min=SIGMA_P_RANGE[0], max=SIGMA_P_RANGE[1], step=0.1,
                             description="P Noise σ:", continuous_update=True)
mu_pr_slider = FloatSlider(value=0, min=PR_RANGE[0], max=PR_RANGE[1], step=0.1,
                           description="Prior:", continuous_update=True)
sigma_pr_slider = FloatSlider(value=6.0, min=SIGMA_PR_RANGE[0], max=SIGMA_PR_RANGE[1], step=0.1,
                              description="Prior σ:", continuous_update=True)
show_post_checkbox = Checkbox(value=False, description="Show Posterior")

# Store last measurement
last_measurement = {"x_obs_p": None}

# Plotting function
def full_bayes(s_p, sigma_p, mu_pr, sigma_pr, show_post, make_m=False):
    fig, ax = plt.subplots(figsize=(8, 6))
    xs = np.linspace(XLIM[0], XLIM[1], 400)
    ax.axvline(s_p, color="grey", linestyle="--")

    # Prior
    var_pr = sigma_pr**2
    pdf_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - mu_pr)**2 / var_pr))
    norm_pr = ((1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - 0)**2 / var_pr))).sum()
    pdf_pr_norm = pdf_pr / norm_pr
    ax.plot(xs, pdf_pr_norm, color="blue", linewidth=2)

    # Measurement
    if make_m:
        rng = np.random.default_rng()
        x_obs_p = s_p + rng.normal(loc=0, scale=sigma_p, size=1)[0]
        last_measurement["x_obs_p"] = x_obs_p
    else:
        x_obs_p = last_measurement["x_obs_p"]

    if x_obs_p is not None:
        var_p = sigma_p**2
        pdf_p = (1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((x_obs_p - xs)**2 / var_p))
        norm_p = ((1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((0 - xs)**2 / var_p))).sum()
        pdf_p_norm = pdf_p / norm_p
        ax.plot(xs, pdf_p_norm, color="red", linewidth=2)

        if show_post:
            s_pm = (var_pr*x_obs_p + var_p*mu_pr)/(var_pr + var_p)
            pdf_post = pdf_p * pdf_pr
            pdf_post_norm = pdf_post / pdf_post.sum()
            ax.plot(xs, pdf_post_norm, color="purple", linewidth=2)
            ax.axvline(s_pm, color="purple", linestyle="--")

    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    ax.set_xlabel("Hand Location")
    ax.set_ylabel("Probability")
    ax.text(0.45 * XLIM[1], 0.9 * YLIM[1], 'Prior Dist.', color="blue", fontsize=12)
    ax.text(0.45 * XLIM[1], 0.83 * YLIM[1], 'Likelihood Dist.', color="red", fontsize=12)
    ax.text(0.45 * XLIM[1], 0.76 * YLIM[1], 'Posterior Dist.', color="purple", fontsize=12)
    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()

# Update function
def update_plot(make_m=False):
    with output:
        output.clear_output(wait=True)
        full_bayes(
            s_p_slider.value,
            sigma_p_slider.value,
            mu_pr_slider.value,
            sigma_pr_slider.value,
            show_post_checkbox.value,
            make_m=make_m
        )

# Button
button = Button(description="Make Measurement", button_style='')

def on_button_click(b):
    update_plot(make_m=True)

button.on_click(on_button_click)

# Observers
s_p_slider.observe(lambda change: update_plot(), names='value')
sigma_p_slider.observe(lambda change: update_plot(), names='value')
mu_pr_slider.observe(lambda change: update_plot(), names='value')
sigma_pr_slider.observe(lambda change: update_plot(), names='value')
show_post_checkbox.observe(lambda change: update_plot(), names='value')

# Layout
ui_left = VBox([
    s_p_slider, sigma_p_slider, mu_pr_slider, sigma_pr_slider,
    show_post_checkbox, button
])
display(HBox([ui_left, output]))
update_plot()
 

## Demo 2B: Multisensory Hand Localization

In the current demo, we will explore Bayesian integration in the case of multisensory proprioception+visual hand localization, 
where the perceived hand location (posterior), is derived from integrating two likelihoods, one for proprioceptive input and 
the other for visual input; a prior can also be added

In the demo, the proprioceptive likelihood is illustrated with a red curve, the visual likelihod is illustrated with a green curve
the prior with a blue curve, and the posterior with a purple curve.

Two adjustable settings control the properties of the Proprioceptive Likelihood distribution.

1. "P Stimulus" controls the location of the hand, sensed via proprioception.
2. "P Noise σ" controls the proprioceptive noise.

Two adjustable settings control the properties of the Visual Likelihood distribution.

1. "V Stimulus" controls the location of the hand, sensed via vision.
2. "V Noise σ" controls the visual noise.

To make a measurement, click the box "Make Measurement". 
Doing so will randomly sample a sensory observation from the measurement distributions and illustrate the Likelihood distributions. 

To illustrate the posterior, click the box "Show Posterior". 

You can also add a prior distribution (click "Add a prior"). The prior mean is set to 0 and the standard deviation is set to 5.


### Guiding Questions
1. Let's first explore what varies from measurement to measurement. Make several measurements. How does this affect each likelihood? 
What about if you change the mean locations and the noise/uncertainty levels?

2. Click to show the posterior distribution. What do you notice about it compared to each likelihood? 
If you change the noise levels, what does this do to the posterior? Explore several combinations, including changing one
    cue at a time. How does the posterior relate to each senosry likelihood?
                               
3. Finally, let's explore what happens if we add a prior. Check this option in the menu. How does change the posterior?
What if you modify the noise levels and stimulus locations?

In [32]:
# ---------------------------------------------------------------------------
# Demo 2B

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Checkbox, VBox, HBox, Button, Output
from IPython.display import display

# Make static images sharp inside notebooks
%matplotlib inline
plt.rcParams["figure.dpi"] = 120

# Axis limits
V_RANGE = (-10, 10)
SIGMA_V_RANGE = (1, 5)
P_RANGE = (-10, 10)
SIGMA_P_RANGE = (1, 5)
XLIM = (-20, 20)
YLIM = (-0.002, 0.05)

# Output widget
output2 = Output()

# Controls
s_v_slider2 = FloatSlider(value=5.0, min=V_RANGE[0], max=V_RANGE[1], step=0.1,
                         description="V Stimulus:", continuous_update=True)
sigma_v_slider2 = FloatSlider(value=2.5, min=SIGMA_V_RANGE[0], max=SIGMA_V_RANGE[1], step=0.1,
                             description="V Noise σ:", continuous_update=True)
s_p_slider2 = FloatSlider(value=-7.0, min=P_RANGE[0], max=P_RANGE[1], step=0.1,
                         description="P Stimulus:", continuous_update=True)
sigma_p_slider2 = FloatSlider(value=4.0, min=SIGMA_P_RANGE[0], max=SIGMA_P_RANGE[1], step=0.1,
                             description="P Noise σ:", continuous_update=True)
show_post_checkbox2 = Checkbox(value=False, description="Show Posterior")
add_prior_checkbox2 = Checkbox(value=False, description="Add a prior")

# Store last measurements
last_measurement = {"x_obs_v": None, "x_obs_p": None}

# Plotting function
def vp_integration(s_v, sigma_v, s_p, sigma_p, show_post, add_prior, make_m2=False):
    fig, ax = plt.subplots(figsize=(8, 6))
    xs = np.linspace(XLIM[0], XLIM[1], 400)

    # Show stimulus locations
    ax.axvline(s_v, color="green", linestyle="--", linewidth=1.5)
    ax.axvline(s_p, color="red", linestyle="--", linewidth=1.5)

    if make_m2:
        rng = np.random.default_rng()
        x_obs_v = rng.normal(loc=s_v, scale=sigma_v, size=1)[0]
        x_obs_p = rng.normal(loc=s_p, scale=sigma_p, size=1)[0]
        last_measurement["x_obs_v"] = x_obs_v
        last_measurement["x_obs_p"] = x_obs_p
    else:
        x_obs_v = last_measurement["x_obs_v"]
        x_obs_p = last_measurement["x_obs_p"]

    if x_obs_v is not None and x_obs_p is not None:
        # Visual likelihood
        var_v = sigma_v**2
        norm_v = ((1 / (np.sqrt(2 * np.pi) * var_v)) * np.exp(-0.5 * ((0 - xs)**2 / var_v))).sum()
        pdf_v = (1 / (np.sqrt(2 * np.pi) * var_v)) * np.exp(-0.5 * ((x_obs_v - xs)**2 / var_v))
        pdf_v_norm = pdf_v / norm_v
        ax.plot(xs, pdf_v_norm, color="green", linewidth=2)

        # Proprioceptive likelihood
        var_p = sigma_p**2
        norm_p = ((1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((0 - xs)**2 / var_p))).sum()
        pdf_p = (1 / (np.sqrt(2 * np.pi) * var_p)) * np.exp(-0.5 * ((x_obs_p - xs)**2 / var_p))
        pdf_p_norm = pdf_p / norm_p
        ax.plot(xs, pdf_p_norm, color="red", linewidth=2)

        if add_prior:
            sigma_pr = 5
            mu_pr = 0
            var_pr = sigma_pr**2
            pdf_pr = (1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - mu_pr)**2 / var_pr))
            norm_pr = ((1 / (np.sqrt(2 * np.pi) * var_pr)) * np.exp(-0.5 * ((xs - 0)**2 / var_pr))).sum()
            pdf_pr_norm = pdf_pr / norm_pr
            ax.plot(xs, pdf_pr_norm, color="blue", linewidth=2)

        if show_post:
            pdf_post = pdf_p * pdf_v
            s_pm = (var_v*x_obs_p + var_p*x_obs_v)/(var_v + var_p)
            if add_prior:
                pdf_post *= pdf_pr
                norm = (var_p**-1 + var_v**-1 + var_pr**-1)
                s_pm = (var_p**-1/norm)*x_obs_p + (var_v**-1/norm)*x_obs_v + (var_pr**-1/norm)*mu_pr
            pdf_post_norm = pdf_post / pdf_post.sum()
            ax.plot(xs, pdf_post_norm, color="purple", linewidth=2)
            ax.axvline(s_pm, color="purple", linestyle="--")

    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    ax.set_xlabel("Hand Location")
    ax.set_ylabel("Probability")
    ax.text(0.45 * XLIM[1], 0.9 * YLIM[1], 'P Likelihood Dist.', color="red", fontsize=12)
    ax.text(0.45 * XLIM[1], 0.83 * YLIM[1], 'V Likelihood Dist.', color="green", fontsize=12)
    ax.text(0.45 * XLIM[1], 0.76 * YLIM[1], 'Posterior Dist.', color="purple", fontsize=12)
    ax.text(0.45 * XLIM[1], 0.69 * YLIM[1], 'Prior Dist.', color="blue", fontsize=12)
    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()

# Update function
def update_plot2(make_m2=False):
    with output2:
        output2.clear_output(wait=True)
        vp_integration(
            s_v_slider2.value,
            sigma_v_slider2.value,
            s_p_slider2.value,
            sigma_p_slider2.value,
            show_post_checkbox2.value,
            add_prior_checkbox2.value,
            make_m2=make_m2
        )

# Button
button2 = Button(description="Make Measurement", button_style='')

def on_button_click(b):
    update_plot2(make_m2=True)

button2.on_click(on_button_click)

# Observers
s_v_slider2.observe(lambda change: update_plot2(), names='value')
sigma_v_slider2.observe(lambda change: update_plot2(), names='value')
s_p_slider2.observe(lambda change: update_plot2(), names='value')
sigma_p_slider2.observe(lambda change: update_plot2(), names='value')
show_post_checkbox2.observe(lambda change: update_plot2(), names='value')
add_prior_checkbox2.observe(lambda change: update_plot2(), names='value')

# Layout
ui_left2 = VBox([
    s_p_slider2, sigma_p_slider2, s_v_slider2, sigma_v_slider2,
    show_post_checkbox2, add_prior_checkbox2, button2
])
display(HBox([ui_left2, output2]))
update_plot2() 